<a href="https://colab.research.google.com/github/marianageh/SWE_MariaNageh_20190403/blob/main/assign4_Train_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v6.1-233-g7433d38 Python-3.7.13 torch-1.11.0+cu113 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.4/107.7 GB disk)


In [4]:
#change directory to the newly cloned YOLOv5 folder
%cd /content/yolov5

/content/yolov5


In [5]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
# %cat {dataset.location}/data.yaml
%cat data.yaml

cat: data.yaml: No such file or directory


In [ ]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
!unzip -q ../custom_data.zip -d ../

In [2]:
# %%writetemplate /content/yolov5/models/custom_yolov5s.yaml
%cd /content
with open('yolov5/models/custom_yolov5s.yaml' , 'w') as f:
    # parameters
    f.write('nc:' + num_classes + '/n')  # number of classes
    f.write('depth_multiple: 0.33' + '/n')  # model depth multiple
    f.write('width_multiple: 0.50' + '/n')  # layer channel multiple
    f.write('/n')
    # anchors
    f.write('anchors:' + '/n')
    f.write('- [10,13, 16,30, 33,23]' + '/n')  # P3/8
    f.write('- [30,61, 62,45, 59,119]' +'/n')  # P4/16
    f.write('- [116,90, 156,198, 373,326]' + '/n')  # P5/32

    # YOLOv5 backbone
    f.write('backbone:' + '/n')
    f.write('   [[-1, 1, Focus, [64, 3]],' + '/n') # 0-P1/2
    f.write('   [-1, 1, Conv, [128, 3, 2]],' + '/n')  # 1-P2/4 
    f.write('   [-1, 3, BottleneckCSP, [128]],' + '/n' )
    f.write('   [-1, 1, Conv, [256, 3, 2]],' + '/n')  # 3-P3/8
    f.write('   [-1, 9, BottleneckCSP, [256]],' + '/n')
    f.write('   [-1, 1, Conv, [512, 3, 2]],' + '/n') # 5-P4/16
    f.write('   [-1, 9, BottleneckCSP, [265]],' + '/n')
    f.write('   [-1, 1, Conv, [1024, 3, 2]],' + '/n')  # 7-P5/32
    f.write('   [-1, 1, SPP, [1024, [5, 9, 13]]],' + '/n')
    f.write('   [-1, 3, BottleneckCSP, [1024]],' + '/n')  # 9
    f.write('   ]' + '/n')

    # YOLOv5 head
    f.write('head:' + '/n')
    f.write('[[-1, 3, BottleNeckCSP, [1024, false]],' + '/n')
    f.write('[-1, 1, Conv, [na * (nc + 5) , 1 ,1 ,0]],' + 'n')
    f.write('[[-2, 1], nn.Upsample, [None , 2, "nearest"]],' + '/n')  # cat backbone P4
    # f.write('[-1, 3, BottleneckCSP, [512, False]],' + '/n')  # 13

    f.write('[[-1 ,6] ,1 ,Concat, [1]]' + '/n')
    f.write('[-1, 1, nn.Conv2d, [512 , 1, 1]],' +'n')
    f.write('[-1, 3, BottleneckCSP, [512, False]],' + '/n')  # 13
    f.write('[-1, 1, nn.Conv2d,[na * (nc*5), 1, 1, 0]]')

    f.write('[-2, 1, nn.Upsample, [None , 2, "nearest"]]' + '/n')
    f.write('[[-1, 4], 1, Concat, [1]],' + '/n')  # cat backbone P3
    f.write('[ -1, 1, Conv, [256, 1, 1]],' + '/n')
    f.write('[-1, 3, BottleneckCSP, [256, False]],' + '/n')  # 17 (P3/8-small)
    f.write('[-1, 1, nn.Conv2d,[na * (nc*5), 1, 1, 0]]')
    f.write('/n')
    f.write('[[] , 1, Detect, [nc,anchors]],' +'n')
    f.write(']' + '/n')
    print('Custom model config is written')

/content


NameError: ignored

In [7]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-233-g7433d38 Python-3.7.13 torch-1.11.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale